In [51]:
conda install selenium

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [52]:
import numpy as numpy
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime
import time

def lazyloading(driver):
    # Get the current number of rows
    current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
    total_number = int(driver.find_elements_by_css_selector('.querySummary__querySummary--39WP2')[0].text.split('Showing ')[1].split('Fortified')[0])
    print("Total Number: ", total_number)
    while current_rows_number < total_number:
        # Scroll down to make new XHR (request more table rows)
        driver.find_element_by_tag_name('body').send_keys(Keys.END)
        try:
            driver.set_window_position(0,-1000)
            # Wait until number of rows increased
            time.sleep(10)
            wait(driver, 20).until(lambda drive: len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0')) > current_rows_number)
            # Update variable with current rows number
            current_rows_number = len(driver.find_elements_by_class_name('explorerCard__explorerCard--3Q7_0'))
            print("Curr Row Number: ", current_rows_number)
        # If number of rows remains the same after 5 seconds passed, break the loop
        # as there no more rows to receive
        except TimeoutException:
            break

In [73]:
from wine import Wine
port_wine_url = 'https://www.vivino.com/explore?e=eJwNizsKgDAQBW_zahUtX-cNxEpE1hhDwKhk4-_2phkGhgmRJYLfWSDIy7opYD72HUxGizNXt_KW6G2SDcfMKMnvTie5bRRncXCxavCkYWRV51uzXAqlkR-x7x-7'
port_wine = Wine(port_wine_url)
port_wine.setup_driver()
port_wine.run_timestamp()
lazyloading(port_wine.driver)
port_wine.run_timestamp()

Current Timestamp:  06-Feb-2020 (08:37:32.928208)
Total Number:  60
Curr Row Number:  50
Curr Row Number:  60
Current Timestamp:  06-Feb-2020 (08:37:53.885047)


In [74]:
port_html = BeautifulSoup(port_wine.driver.page_source, 'lxml')
port_div = port_html.find("div", {"class": "explorerPage__results--3wqLw"})
port_rows = port_html.find_all("div", {"class": "explorerCard__explorerCard--3Q7_0"})

In [75]:
len(port_rows)

60

In [76]:
import copy

# austria_wine_df = pd.DataFrame()
all_rows = []

# Let's store each row as a dictionary 
empty_row = {
    "title": None, "location": None, "country": None, "price": None, "type": None, "ratings": None, "num_ratings": None, "reviews": None, "url": None
}

for row in port_rows:
    new_row = copy.copy(empty_row)
    # A list of all the entries in the row.
    new_row['title'] = row.find("span", {"class": "vintageTitle__wine--U7t9G"}).text
    location = row.find("div", {"class": "vintageLocation__vintageLocation--1DF0p"})
    new_row['location'] = location.findChildren()[-1].text
    country = row.find("i", {"class": "vintageLocation__countryFlag--1HbXr"})['title']
    new_row['country'] = country
    price_button = row.find("button", {"class": "addToCartButton__addToCartButton--qZv9F"})
    if price_button:
        new_row['price'] = (float(price_button.find("span").text.replace("$", "")))
    new_row['type'] = 'Port'
    new_row['ratings'] = row.find("div", {"class": "vivinoRatingWide__averageValue--1zL_5"}).text
    new_row['num_ratings'] = int(row.find("div", {"class": "vivinoRatingWide__basedOn--s6y0t"}).text.split()[0])
    review_div = row.find("div", {"class": "review__note--2b2DB"})
    if review_div:
        new_row['reviews'] = review_div.text
    clean_div = row.find("div", {"class": "cleanWineCard__cleanWineCard--tzKxV cleanWineCard__row--CBPRR"})
    if clean_div:
        new_row['url'] = 'https://www.vivino.com' + clean_div.find("a")['href']
    all_rows.append(new_row)

In [77]:
all_rows

[{'title': 'Ink Grade Port N.V.',
  'location': 'Napa Valley',
  'country': 'United States',
  'price': 37.99,
  'type': 'Port',
  'ratings': '4.2',
  'num_ratings': 587,
  'reviews': None,
  'url': 'https://www.vivino.com/heitz-ink-grade-port/w/83245?year=N.V.&price_id=18402695'},
 {'title': 'Aged 10 Years Tawny Port N.V.',
  'location': 'Madera',
  'country': 'United States',
  'price': 31.99,
  'type': 'Port',
  'ratings': '4.2',
  'num_ratings': 47,
  'reviews': None,
  'url': 'https://www.vivino.com/ficklin-vineyards-aged-10-years-tawny-port/w/11291?year=N.V.&price_id=4357081'},
 {'title': 'Port 2012',
  'location': 'Napa Valley',
  'country': 'United States',
  'price': 49.99,
  'type': 'Port',
  'ratings': '4.2',
  'num_ratings': 35,
  'reviews': None,
  'url': 'https://www.vivino.com/prager-winery-and-port-works-napa-valley-port/w/1303324?year=2012&price_id=18479255'},
 {'title': 'Petite Sirah Port 2015',
  'location': 'Clarksburg',
  'country': 'United States',
  'price': 19.9

In [17]:
conda install pymongo

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [78]:
import pymongo
from pymongo import MongoClient

client = MongoClient('localhost', 27017)
wine_db = client['wine_db']
wines = wine_db['wines']

In [79]:
for row in all_rows:
    wines.insert_one(row)